# Fase 6: Resolución numérica de la Ecuación diferencial de Black-Scholes-Merton

Implementar un esquema de diferencias finitas implícitas para resolver la ecuación
diferencial de Black-Scholes-Merton, en el caso de una PUT europea y una opción binaria.

In [1]:
#Importo mis librerias
import numpy as np
import matplotlib.pyplot as plt
import Funciones

### Parámetros iniciales

In [2]:
# Defino los parámetros iniciales
S0 = 112      # Precio inicial del activo
K = 115       # Precio de ejercicio
r = 0.08      # Tasa libre de riesgo
sigma = 0.18  # Volatilidad
T = 1         # Tiempo a vencimiento (en años)

## Discretización del tiempo y riqueza

Definimos los parámetros de la malla y posteriormente generamos la malla de tiempo y riqueza.

\begin{equation*}
\Delta t = \frac{T}{N}, \quad \Delta S = \frac{S_{max}}{M}
\end{equation*}

In [3]:
#Parámetros de la malla
S_max= 2*K  # Precio máximo del activo
M= 100  # Número de puntos en la malla de precios
N = 100  # Número de puntos en la malla de tiempo
dS=S_max/M  # Tamaño del paso en la malla de precios
dT = T/N  # Tamaño del paso en la malla de tiempo

## Implementar condiciones de frontera apropiadas

Generamos la malla de tiempo y riqueza, y luego definimos la condición de frontera al vencimiento de la opción PUT europea que es:
\begin{equation*}
V(S, T) = \max(K - S, 0)   
\end{equation*}
Y para la opción binaria:
\begin{equation*}
V(S, T) = \begin{cases}
1 & \text{si } S < K \\
0 & \text{si } S \geq K
\end{cases}
\end{equation*}

Además definimos las condiciones de frontera para $S = 0$ y $S = S_{max}$:
\begin{equation*}
V(0, t) = K e^{-r(T-t)}, \quad V(S_{max}, t) = 0
\end{equation*}

In [4]:
#Genramos las mallas para PUT y binario
V_put = Funciones.inicializar_malla(M, K,S_max, tipo='put')
V_binario = Funciones.inicializar_malla(M, K,S_max, tipo='binario')

#Añadimos las condiciones de frontera
V_put[0]=K*np.exp(-r*T)  # Condición de frontera inferior para PUT
V_put[-1] = 0  # Condición de frontera superior para PUT

V_binario[0]=np.exp(-r*T)  # Condición de frontera inferior para binario
V_binario[-1] = 0  # Condición de frontera superior para binario

## Solucionar la ecuación diferencial parcial

Implementamos el esquema de diferencias finitas implícitas para resolver la ecuación diferencial de Black-Scholes-Merton. Definimos los coeficientes del sistema de la siguiente manera:
\begin{equation*}
\alpha= \Delta t \left( \frac{a}{\Delta S^2} - \frac{b}{2 \Delta S} \right), \quad
\beta = 1 - \Delta t \left( \frac{a}{\Delta S^2} + c \right), \quad
\gamma = \Delta t \left( \frac{a}{\Delta S^2} + \frac{b}{2 \Delta S} \right)
\end{equation*}

Donde $a = \frac{1}{2} \sigma^2 S^2$, $b = r S$, y $c = r$.

Posterior a esto, construimos la matriz del sistema y resolvemos el sistema lineal en cada paso de tiempo hacia atrás.

In [5]:
#Matriz tridiagonal
A = Funciones.crear_matriz_tridiagonal(M, dS, dT, sigma, r)

#Calculamos las soluciones
put_sol= Funciones.solucion_implicita(dT,A,V_put,K,r,dS,S0,T,N,tipo='put')
bin_sol= Funciones.solucion_implicita(dT,A,V_binario,K,r,dS,S0,T,N,tipo='binario')

## Calcular el error

Calcular el error en términos del tamaño de las subdiviciones de $\Delta t$ y $\Delta S$ de la
discretización.

In [6]:
valoresMN=[(10,10), (20,20), (50,50), (100,100), (200,200), (500,500), (1000,1000)]
errores_put = Funciones.calcular_errores(S0, K, r, sigma, T, valoresMN,S_max,tipo='put')
errores_binario = Funciones.calcular_errores(S0, K, r, sigma, T, valoresMN,S_max,tipo='binario')

for M, N, num, ref, err in errores_put:
    print(f"M={M}, N={N} -> Valor numérico: {num:.4f}, Valor exacto: {ref:.4f}, Error: {err:.4e}")

for M, N, num, ref, err in errores_binario:
    print(f"M={M}, N={N} -> Valor numérico: {num:.4f}, Valor exacto: {ref:.4f}, Error: {err:.4e}")

M=10, N=10 -> Valor numérico: 4.2935, Valor exacto: 11.0860, Error: 6.7925e+00
M=20, N=20 -> Valor numérico: 5.1062, Valor exacto: 11.0860, Error: 5.9798e+00
M=50, N=50 -> Valor numérico: 5.2174, Valor exacto: 11.0860, Error: 5.8686e+00
M=100, N=100 -> Valor numérico: 5.2331, Valor exacto: 11.0860, Error: 5.8529e+00
M=200, N=200 -> Valor numérico: 5.2398, Valor exacto: 11.0860, Error: 5.8462e+00
M=500, N=500 -> Valor numérico: 5.2427, Valor exacto: 11.0860, Error: 5.8433e+00
M=1000, N=1000 -> Valor numérico: 5.2435, Valor exacto: 11.0860, Error: 5.8425e+00
M=10, N=10 -> Valor numérico: 0.1747, Valor exacto: 0.3857, Error: 2.1091e-01
M=20, N=20 -> Valor numérico: 0.2927, Valor exacto: 0.3857, Error: 9.2996e-02
M=50, N=50 -> Valor numérico: 0.4314, Valor exacto: 0.3857, Error: 4.5737e-02
M=100, N=100 -> Valor numérico: 0.4079, Valor exacto: 0.3857, Error: 2.2237e-02
M=200, N=200 -> Valor numérico: 0.3966, Valor exacto: 0.3857, Error: 1.0990e-02
M=500, N=500 -> Valor numérico: 0.3820, Val